# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [14]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP, temperature, alertness, inspired_oxygen, performance_status):


 '''This function's initial section determines an individual's COVID-19 severity based on their vital signs and a few other factors.
    The MDCalc website provides the algorithm that was employed.
    We can then forecast the likelihood of any negative effects the patient may experience over the course of the next 30 days based on the score.

     >>> priest('FEMALE',40,24,0.96,105,115,39.9,'Alert','AIR','Unrestricted normal activity')
     0.09
     >>> priest('MaLe',32,20,0.93,100,110,32.2,'CONFUSED OR NOT ALERT','SUPPLEMENTAL OXYGEN','LIMITED SELF-CARE')
     0.55
     >>> priest ('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no SELF-CARE')
     0.26
     >>> priest('MalE',65,24,0.87,90,99,36.5,'ConFusEd oR nOt AleRt','air','limited strenous activity, can do light activity')
     0.47
     >>> priest('male',50,20,0.90,87,95,35,'alert','supplemental oxygen','bed/chair bound, no self-care')
     0.59
     '''

 sex = sex.lower()
 if sex not in ["male", "female"]:
  return None
 if sex == "male":
  sex_score = 1
 elif sex == "female":
  sex_score = 0

 if age >= 16 and age <= 49:
   age_score = 0
 elif age >=50 and age <= 65:
  age_score = 2
 elif age >=66 and age <= 80:
  age_score = 3
 elif age > 80:
  age_score = 4
 if respiratory_rate < 9:
  respiratory_rate_score = 3
 elif respiratory_rate >= 9 and respiratory_rate <= 11:
  respiratory_rate_score = 1
 elif respiratory_rate >= 12 and respiratory_rate <= 20:
  respiratory_rate_score = 0
 elif respiratory_rate >= 21 and respiratory_rate <= 24:
  respiratory_rate_score = 2
 elif respiratory_rate > 24:
  respiratory_rate_score = 3
 if oxygen_saturation > 0.95:
  oxygen_saturation_score = 0
 elif oxygen_saturation >= 0.94 and oxygen_saturation <= 0.95:
  oxygen_saturation_score = 1
 elif oxygen_saturation >= 0.92 and oxygen_saturation <=0.93:
  oxygen_saturation_score = 2
 elif oxygen_saturation < 0.92:
  oxygen_saturation_score = 3
 if heart_rate < 41:
  heart_rate_score = 3
 elif heart_rate >= 41 and heart_rate <= 50:
  heart_rate_score = 1
 elif heart_rate >= 51 and heart_rate <= 90:
  heart_rate_score = 0
 elif heart_rate >= 91 and heart_rate <= 110:
  heart_rate_score = 1
 elif heart_rate >= 111 and heart_rate <= 130:
  heart_rate_score = 2
 elif heart_rate > 130:
  heart_rate_score = 3
 if systolic_BP < 91:
  systolic_BP = 3
 elif systolic_BP >= 91 and systolic_BP <= 100:
  systolic_BP = 2
 elif systolic_BP >= 101 and systolic_BP <= 110:
  systolic_BP = 1
 elif systolic_BP >= 111 and systolic_BP <= 219:
  systolic_BP = 0
 elif systolic_BP > 219:
  systolic_BP = 3
 if temperature < 35.1:
  temperature_score = 3
 elif temperature >= 35.1 and temperature <= 36.0:
  temperature_score = 1
 elif temperature >= 36.1 and temperature <= 38.0:
  temperature_score = 0
 elif temperature >= 38.1 and temperature <= 39.0:
  temperature_score = 1
 elif temperature > 39.0:
  temperature_score = 2
 alertness = alertness.lower()
 if alertness == "alert":
  alertness_score = 0
 elif alertness == "confused or not alert":
  alertness_score = 3
 if inspired_oxygen.lower() == "air":
  inspired_oxygen_score = 0
 elif inspired_oxygen.lower() == "supplemental oxygen":
  inspired_oxygen_score = 2
 if performance_status.lower() == "unrestricted normal activity":
  performance_status_score = 0
 elif performance_status.lower() == "limited strenous activity, can do light activity":
  performance_status_score = 1
 elif performance_status.lower() == "limited activity, can self-care":
  performance_status_score = 2
 elif performance_status.lower() == "limited self-care":
  performance_status_score = 3
 elif performance_status.lower() == "bed/chair bound, no self-care":
  performance_status_score = 4


 risk_score = sum([sex_score, age_score, respiratory_rate_score, oxygen_saturation_score, heart_rate_score, systolic_BP, temperature_score, alertness_score, inspired_oxygen_score, performance_status_score])
 if risk_score >= 0 and risk_score <= 1:
  risk = 0.01
 elif risk_score >=2 and risk_score <= 3:
  risk = 0.02
 elif risk_score == 4:
  risk = 0.03
 elif risk_score == 5:
  risk = 0.09
 elif risk_score == 6:
  risk = 0.15
 elif risk_score == 7:
  risk = 0.18
 elif risk_score == 8:
  risk = 0.22
 elif risk_score == 9:
  risk = 0.26
 elif risk_score == 10:
  risk = 0.29
 elif risk_score == 11:
  risk = 0.29
 elif risk_score == 12:
  risk = 0.38
 elif risk_score == 13:
  risk = 0.46
 elif risk_score == 14:
  risk = 0.47
 elif risk_score == 15:
  risk = 0.49
 elif risk_score == 16:
  risk = 0.55
 elif risk_score >= 17 and risk_score <= 25:
  risk = 0.59
 elif risk_score >= 26:
  risk = 0.99
 return risk

In [15]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('FEMALE',40,24,0.96,105,115,39.9,'Alert','AIR','Unrestricted normal activity')
Expecting:
    0.09
ok
Trying:
    priest('MaLe',32,20,0.93,100,110,32.2,'CONFUSED OR NOT ALERT','SUPPLEMENTAL OXYGEN','LIMITED SELF-CARE')
Expecting:
    0.55
ok
Trying:
    priest ('FemalE', 22, 17, 0.98, 96, 103, 31.0, 'AleRT', 'AiR', 'BED/CHAIR bOuNd, no SELF-CARE')
Expecting:
    0.26
ok
Trying:
    priest('MalE',65,24,0.87,90,99,36.5,'ConFusEd oR nOt AleRt','air','limited strenous activity, can do light activity')
Expecting:
    0.47
ok
Trying:
    priest('male',50,20,0.90,87,95,35,'alert','supplemental oxygen','bed/chair bound, no self-care')
Expecting:
    0.59
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests
import json
def find_hospital(age, sex, risk):
  '''This function analyzes the patient's age, sex, and risk score to identify which nearby facility to take them to in an emergency.
  It does this by using the REST web service.

  >>> find_hospital(45, 'male', 0.2)
  'Southwest Hospital and Medical Center'
  >>> find_hospital(65, 'male', 0.49)
  'Wesley Woods Geriatric Hospital'
  >>> find_hospital(29, 'female', 0.56)
  'Emory Dunwoody Medical Center'
  >>> find_hospital(19, 'female', 0.16)
  'Select Specialty Hospital - Northeast Atlanta'
  >>> find_hospital(22, 'female', 0.92)
  'Emory Dunwoody Medical Center'
  '''
  url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
  response = requests.get(url)
  hospital_name = response.json()['hospital']
  return hospital_name

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(45, 'male', 0.2)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(65, 'male', 0.49)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(29, 'female', 0.56)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(19, 'female', 0.16)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok
Trying:
    find_hospital(22, 'female', 0.92)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [ ]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address for the given hospital name from a JSON file.

    Args:
    hospital_name (str): Name of the hospital.

    Returns:
    str or None: The address of the hospital. Returns None if the hospital name is not found.
    >>> get_address('COOK MEDICAL CENTER')
    '706 NORTH PARRISH AVENUE'

    >>> get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
    '417 THIRD AVENUE'

    >>> get_address("WALTON REHABILITATION HOSPITAL")
    '1355 INDEPENDENCE DRIVE'

    """
    # URL for the JSON data
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    # Send a GET request to retrieve the JSON data
    response = requests.get(json_url)

    # Parse the JSON data if the response status code is 200 (OK)
    if response.status_code == 200:
        data = response.json()

        # Search for the hospital by name
        if hospital_name in data:
            return data[hospital_name]["ADDRESS"]

    # Return None if any part of the process fails
    return None

In [5]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address for the given hospital name from a JSON file.

    Args:
    hospital_name (str): Name of the hospital.

    Returns:
    str or None: The address of the hospital. Returns None if the hospital name is not found.
    >>> get_address('COOK MEDICAL CENTER')
    '706 NORTH PARRISH AVENUE'

    >>> get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
    '417 THIRD AVENUE'

    >>> get_address("WALTON REHABILITATION HOSPITAL")
    '1355 INDEPENDENCE DRIVE'

    """
    # URL for the JSON data
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    # Send a GET request to retrieve the JSON data
    response = requests.get(json_url)

    # Parse the JSON data if the response status code is 200 (OK)
    if response.status_code == 200:
        data = response.json()

        # Search for the hospital by name
        if hospital_name in data:
            return data[hospital_name]["ADDRESS"]

    # Return None if any part of the process fails
    return None

In [6]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('COOK MEDICAL CENTER')
Expecting:
    '706 NORTH PARRISH AVENUE'
ok
Trying:
    get_address('PHOEBE PUTNEY MEMORIAL HOSPITAL')
Expecting:
    '417 THIRD AVENUE'
ok
Trying:
    get_address("WALTON REHABILITATION HOSPITAL")
Expecting:
    '1355 INDEPENDENCE DRIVE'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [19]:
def process_people(file_location):
    response = requests.get(file_location)
    data = response.text.split('\n')
    result = {}

    for line in data[1:]:
        if line.strip():  # Skips empty lines
            row = line.strip().split("|")
            patient_number = row[0]
            sex = row[1].capitalize()
            age = int(row[2])
            breath = int(row[3])
            o2sat = float(row[4])
            heart = int(row[5])
            systolic = int(row[6])
            temp = float(row[7])
            alertness = row[8].capitalize()
            inspired = row[9].capitalize()
            status = row[10].capitalize()

            # Call the risk_calculator function and other necessary functions to get the hospital and address
            risk_score = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)
            hospital = find_hospital(age, sex, risk_score)
            address = get_address(hospital)

            # Update the result dictionary
            result[patient_number] = [
                sex,
                age,
                breath,
                o2sat,
                heart,
                systolic,
                temp,
                alertness,
                inspired,
                status,
                risk_score,
                hospital,
                address
            ]

    return result


In [9]:
import requests
import json

def risk_calculator(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status):
    """
    Calculate the risk, hospital, and address based on the provided parameters.

    Args:
        sex (str): Gender assigned at birth, 'male' or 'female'.
        age (int): Age in years.
        breath (int): Respiratory rate in breaths per minute.
        o2sat (float): Oxygen saturation, a percent as a float between 0 and 1.
        heart (int): Heart rate in beats per minute.
        systolic (int): Systolic blood pressure in mmHg.
        temp (float): Temperature in degrees Celsius.
        alertness (str): Description of alertness, e.g., 'alert'.
        inspired (str): Description of oxygen source, e.g., 'air'.
        status (str): Description of physical activity, e.g., 'unrestricted normal activity'.

    Returns:
        tuple: A tuple containing risk score (float), hospital name (str), and hospital address (str).
    """
    # Placeholder implementation for risk calculation
    risk_score = 0.0

    # Placeholder implementation for hospital and address
    hospital_name = find_hospital(age, sex, risk_score)
    hospital_address = get_address(hospital_name)

    return risk_score, hospital_name, hospital_address

def find_hospital(age, sex, risk_score):
    """
    Find the hospital based on the patient's age, sex, and risk score.

    Args:
        age (int): The age of the patient.
        sex (str): The sex of the patient ('male' or 'female').
        risk_score (float): The calculated risk score.

    Returns:
        str: The name of the hospital.
    """
    try:
        url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_score}'
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        hospital_name = data.get('hospital')
        return hospital_name
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def get_address(hospital_name):
    """
    Retrieve the address of a hospital based on its name from a JSON file.

    Args:
        hospital_name (str): The name of the hospital to search for.

    Returns:
        str or None: The address of the hospital if found, or None if not found.
    """
    try:
        # Define the URL for the JSON file
        json_url = 'https://hds5210-data.s3.amazonaws.com/ga_hospitals.json'

        # Send a GET request to download the JSON data
        response = requests.get(json_url)

        # Check if the request was successful
        response.raise_for_status()

        # Load JSON data
        hospital_data = json.loads(response.text)

        # Search for the hospital by name (case-insensitive)
        for key, value in hospital_data.items():
            if hospital_name.lower() in key.lower():
                return value.get('ADDRESS')

        # If the hospital name is not found, return None
        return None
    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def process_people(file_location):
    """
    Process the people.psv file and calculate the risk for each individual.

    Args:
        file_location (str): The URL of the .psv file containing patient information.

    Returns:
        list: A list of dictionaries where each dictionary represents a patient's information along with calculated risk.
    """
    response = requests.get(file_location)
    data = response.text.split('\n')
    result = {}

    for line in data[1:]:
        if line.strip():  # Skips empty lines
            row = line.strip().split("|")
            patient_number = row[0]
            sex = row[1].upper()
            age = int(row[2])
            breath = int(row[3])
            o2sat = float(row[4])
            heart = int(row[5])
            systolic = int(row[6])
            temp = float(row[7])
            alertness = row[8].upper()
            inspired = row[9].upper()
            status = row[10].upper()

            # Call risk_calculator function
            risk_score, hospital, address = risk_calculator(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

            if patient_number in result:
                result[patient_number].append([sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address])
            else:
                result[patient_number] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]

    return result

# Example usage:
file_location = 'https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh'
result = process_people(file_location)

# Save the results to a JSON file
with open('your_results.json', 'w') as output_file:
    json.dump(result, output_file, indent=4)

print(result)

{'E9559': ['FEMALE', 40, 24, 0.96, 105, 115, 34.9, 'ALERT', 'AIR', 'UNRESTRICTED NORMAL ACTIVITY', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9385': ['FEMALE', 51, 19, 0.99, 90, 91, 38.1, 'ALERT', 'SUPPLEMENTAL OXYGEN', 'UNRESTRICTED NORMAL ACTIVITY', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E3067': ['FEMALE', 40, 29, 0.96, 105, 95, 38.1, 'ALERT', 'AIR', 'UNRESTRICTED NORMAL ACTIVITY', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9422': ['FEMALE', 66, 19, 0.96, 135, 115, 38.1, 'ALERT', 'AIR', 'BED/CHAIR BOUND, NO SELF-CARE', 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE'], 'E8661': ['MALE', 18, 22, 0.95, 105, 115, 36.3, 'CONFUSED OR NOT ALERT', 'AIR', 'UNRESTRICTED NORMAL ACTIVITY', 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW'], 'E6235': ['MALE', 16, 23, 0.96, 135, 115, 39.2, 'ALERT', 'SUPPLEMENTAL OXYGEN', 'UNRESTRICTED NORMAL ACTIVITY', 'Childrens Healthcar

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import requests
import json

# Define the URL for the JSON file
json_url = 'https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5'

# Send a GET request to download the JSON data
response = requests.get(json_url)

# Check if the request was successful
if response.status_code == 200:
    # Load the JSON data
    expected_results = json.loads(response.text)

    # Assuming you have your results stored in the 'result' variable, compare it with the expected results
    # Make sure the 'result' variable contains the same format as the 'expected_results' variable
    # Compare the dictionaries to ensure the results match

    # Example comparison
    if result == expected_results:
        print("Results match the expected results!")
    else:
        print("Results do not match the expected results. There might be an error in your code.")
else:
    print("Failed to fetch JSON data. Please check the URL or your internet connection.")


Results do not match the expected results. There might be an error in your code.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---